<h1><center>МЕТОДЫ ВЕРСИОНИРОВАНИЯ</h1></center>

Прежде чем переходить к работе с MLflow API, убедитесь, что вам доступна сама модель, то есть она зарегистрирована в реестре моделей MLflow, и поднят MLflow. Если это не так, то:
- Поднимите MLflow, используя предложенный ранее вариант конфигурации: локальный Tracking Server, удалённое хранилище для экспериментов и хранилище для артефактов.
- Добавьте модель в реестр моделей.

Теперь обучите новую версию модели. После этого обновите версию модели в реестре и не забудьте посчитать метрики: ROC-AUC, precision, recall и другие, чтобы сравнивать модели между собой.

In [3]:
import pandas as pd
import numpy as np
import random
import os
import joblib

import mlflow
from mlflow.tracking import MlflowClient

import catboost as cb
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

from category_encoders import CatBoostEncoder

**Обучим модель**. Загрузим и подготовим данные:


In [4]:
df = pd.read_csv('users_churn.csv')
print('Колво строк до обработки:',df.shape[0])

# Закодируем целевую переменную
df['target'] = (df['end_date'].notna()).astype(int)

# Выделим признаки в три отдельные таблицы для дальнейшей работы:
features = df.drop(['customer_id','target'],axis=1)
num_features = features.select_dtypes(include=['float', 'int'])
date_features = features.select_dtypes(include='datetime64[ns]')
cat_features = features.select_dtypes(include='object')

# Посчитаем колво уникальных значений для катег. переменных и создадим создадим два датасета:
unique_values_per_col = cat_features.nunique().value_counts()
binary_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()==2] ]
other_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()!=2] ]

# Бинарные подразделяем на два - "да/нет" и другие бинарные:
yes_no_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()==True] ]
other_binary_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()!=True] ]

# Дубликаты
is_duplicated_id = df.duplicated(subset=['customer_id'], keep=False)

# Пропуски
cols_with_nans = df.isnull().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
for col in cols_with_nans:
    if df[col].dtype in [float, int]:
        fill_value = df[col].mean()
    elif df[col].dtype == 'object':
        fill_value = df[col].mode().iloc[0]
    df[col] = df[col].fillna(fill_value)

# Выбросы
num_cols = df.select_dtypes(['float']).columns
threshold = 1.5
potential_outliers = pd.DataFrame()
for col in num_cols:
	Q1 = df[col].quantile(0.25)
	Q3 = df[col].quantile(0.75)
	IQR = Q3 - Q1
	margin = threshold * IQR
	lower = Q1 - margin
	upper = Q3 + margin
	potential_outliers[col] = ~df[col].between(lower, upper)
outliers = potential_outliers.any(axis=1)

df.drop(columns=['id', 'customer_id', 'begin_date', 'end_date'], inplace=True)
df[-3:]


Колво строк до обработки: 7019


,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
7016,Month-to-month,Yes,Electronic check,29.60,346.45,DSL,Yes,No,No,No,No,No,Female,0,Yes,Yes,No,0
7017,Month-to-month,Yes,Mailed check,74.40,306.60,Fiber optic,No,No,No,No,No,No,Male,1,Yes,No,Yes,1
7018,Two year,Yes,Bank transfer (automatic),105.65,6844.50,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Male,0,No,No,No,0


Обучим модель:

In [5]:

# Разделим данные на две части - для обучения и для проверки качества предсказания:
X_tr, X_val, y_tr, y_val = train_test_split(df, df['target'], stratify=df['target']) 

# Тренировочная выборка
cat_features_tr = X_tr.select_dtypes(include='object')
potential_binary_features_tr = cat_features_tr.nunique() == 2

binary_cat_features_tr = cat_features_tr[potential_binary_features_tr[potential_binary_features_tr].index]
other_cat_features_tr = cat_features_tr[potential_binary_features_tr[~potential_binary_features_tr].index]
num_features_tr = X_tr.select_dtypes(['float'])

# Валидационная выборка
cat_features_val = X_val.select_dtypes(include='object')
potential_binary_features_val = cat_features_val.nunique() == 2

binary_cat_features_val = cat_features_val[potential_binary_features_val[potential_binary_features_val].index]
other_cat_features_val = cat_features_val[potential_binary_features_val[~potential_binary_features_val].index]
num_features_val = X_val.select_dtypes(['float'])

binary_cols = binary_cat_features_tr.columns.tolist()
non_binary_cat_cols = other_cat_features_tr.columns.tolist()
num_cols = num_features_tr.columns.tolist()

# Определим список трансформаций в рамках ColumnTransformer
preprocessor = ColumnTransformer( [ ('binary', OneHotEncoder(drop='if_binary'), binary_cols),
                                    ('cat', CatBoostEncoder(), non_binary_cat_cols),
                                    ('num', StandardScaler(), num_cols) ],verbose_feature_names_out=False )

# Трансформируем исходные данные с помощью созданного preprocessor
X_tr_transformed = preprocessor.fit_transform(X_tr, y_tr)
X_val_transformed = preprocessor.transform(X_val)

# Создадим модель:
model = CatBoostClassifier()

# Обучим модель:
model.fit(X_tr_transformed, y_tr)

Learning rate set to 0.020938
0:	learn: 0.6792724	total: 95.5ms	remaining: 1m 35s
1:	learn: 0.6674446	total: 98.2ms	remaining: 49s
2:	learn: 0.6551460	total: 101ms	remaining: 33.5s
3:	learn: 0.6439674	total: 103ms	remaining: 25.7s
4:	learn: 0.6326817	total: 106ms	remaining: 21.1s
5:	learn: 0.6220767	total: 109ms	remaining: 18s
6:	learn: 0.6124227	total: 111ms	remaining: 15.8s
7:	learn: 0.6039995	total: 114ms	remaining: 14.1s
8:	learn: 0.5940092	total: 116ms	remaining: 12.8s
9:	learn: 0.5859916	total: 119ms	remaining: 11.7s
10:	learn: 0.5770932	total: 121ms	remaining: 10.9s
11:	learn: 0.5687058	total: 124ms	remaining: 10.2s
12:	learn: 0.5605243	total: 127ms	remaining: 9.63s
13:	learn: 0.5539383	total: 129ms	remaining: 9.11s
14:	learn: 0.5478143	total: 132ms	remaining: 8.66s
15:	learn: 0.5408693	total: 134ms	remaining: 8.27s
16:	learn: 0.5351941	total: 137ms	remaining: 7.92s
17:	learn: 0.5297893	total: 140ms	remaining: 7.61s
18:	learn: 0.5247935	total: 142ms	remaining: 7.34s
19:	learn: 0

Проверим качество нашей модели:

In [6]:
# Сделаем предсказание:
y_val_pred = model.predict(X_val_transformed)

# Определим точность:
accuracy = str(accuracy_score(y_val, y_val_pred))[:4]
print(f"Точность модели: {accuracy}")

Точность модели: 0.80


In [9]:
EXPERIMENT_NAME = "churn_nikolaistepanov"
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_nikolaistepanov"

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" # ваш код здесь
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE3Nlz8iDILW5VTYM1ihQB' # ваш код здесь
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44' # ваш код здесь

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

pip_requirements="../requirements.txt" # ваш код здесь
signature = mlflow.models.infer_signature(X_val, y_val_pred) # ваш код здесь
input_example = X_val[:10] # ваш код здесь
metadata = {'model_type':'monthly'} # ваш код здесь
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
     # ваш код логирования метрик здесь
    model_info = mlflow.catboost.log_model( model, cb_model=model,
                                           artifact_path='models',
                                           registered_model_name=REGISTRY_MODEL_NAME,
                                           pip_requirements=pip_requirements,
                                           signature=signature,
                                           input_example=input_example,
                                           metadata=metadata,
                                           code_paths=code_paths,
                                           await_registration_for=60 ) 

    mlflow.log_metrics(metrics)
    # ваш код логирования модели здесь

/home/mle-user/mle_projects/mle-mlflow/.venv/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None


AttributeError: 'NoneType' object has no attribute 'experiment_id'